In [9]:
import os
import logging
from dotenv import load_dotenv

# For database connection
from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session
# Load environment variables from .env file in the subdirectory named app
load_dotenv(os.path.join(os.getcwd(), 'app', '.env'))

# Import the verified_query module
#from app.helper import (
#    enhance_question
#)
from app.models.verified_query import VerifiedQuery, Question
from app.utilities import config

# For LLM service
from app.llm.llm_service import LLMService

# Set up logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configure database connections
engine = create_engine(config.APPLICATION_DB_CONNECTION_STRING)
insurance_db_engine = create_engine(config.BUSINESS_DB_CONNECTION_STRING)

# Initialize LLM service
llm_service = LLMService()

In [21]:
def enhance_question(question: str, context, llm_service) -> str:
    """
    Enhance a user question using LLM to make it more specific and clear.
    
    Args:
        question: User question
        llm_service: LLM service for generating enhanced question
    Returns:
        Enhanced question
    """

    # Create a system prompt for the LLM
    system_prompt = """You are an expert in P&C Insurance data analysis.
    Your task is to enhance user questions to make them more specific and clear."""

    # User prompt with the original question
    user_prompt = f"""
    You are helping clarify and enhance user questions for insurance data analysis. Your goal is to make them more specific and clear.

    Below is a user question the user wants answered using data. Use the rules below to enhance it.

    User Question:
    {question}

    Enhancement Rules:

    RULE #1: Use the calendar context below ONLY if the original question makes reference to it (e.g., "this year", "last 2 quarters", etc). DO NOT assume or add time period, unless clearly implied by the original question.
    Calendar Context: {context.get('calendar_context', 'None')}
    
    RULE #2: Use the user profile context below ONLY if the original question makes reference to it (e.g., "my region", "my agency", etc). DO NOT use user profile information unless clearly implied by the original question.
    User Profile: {context.get('user_profile', 'None')}

    RULE #3: If the question is general, keep it general. If no enhancement is needed, return the original question as is.
    
    RULE #4: Preserve the intent, structure, and tone of the original question.
    
    RULE #5: Return only the enhanced question with no extra commentary.

    If your response does not pass the above rules, please rephrase it to comply with the rules.

    Examples:
    - Input: "What is the total premium generated by each agency?" → Output: "What is the total premium generated by each agency?"
    - Input: "What is the total premium generated by each agency in my region this year?" (with Region: Northeast) → Output: "What is the total premium generated by each agency in the Northeast region during the current year (2025)?"
    """

    try:
        logger.info("Enhancing question with LLM...")
        
        # Get enhanced question from LLM
        enhanced_question = llm_service.generate_text(
            prompt=user_prompt,
            system_prompt=system_prompt,
            temperature=0
        )
        # Strip any leading/trailing whitespace
        enhanced_question = enhanced_question.strip()
    except Exception as e:
        logger.error(f"Error generating enhanced question: {str(e)}")
        
    return enhanced_question
# Add some sample context
context = {
    "calendar_context": "current date is 2025-04-30, current year is 2025, previous year is 2024, current quarter is 2025 Q2, previous quarter is 2025 Q1, current month is 2025-04, previous month is 2025-03",
    "user_profile": "'my region' is 'Northeast'",
}

enhanced = {}
questions = [
    "What is the total premium generated by each agency?",
    "What is the total premium generated by each agency in my region this year?",
    "Which agencies generated the most premium in my region this year?",
    "How does the premium generated this year compare to last year?",
    "What is the trend of premium generation over the last five years?",
    "Which agencies have shown the most growth in premium generation?",
    "What are the top 5 agencies by premium generation this quarter?",
]
for question in questions:
    enhanced[question] = enhance_question(question, context, llm_service)

# Print the enhanced questions
for question, enhanced_question in enhanced.items():
    print(f"Original: {question}")
    print(f"Enhanced: {enhanced_question}")
    print("")

Original: What is the total premium generated by each agency?
Enhanced: What is the total premium generated by each agency during the current year (2025)?

Original: What is the total premium generated by each agency in my region this year?
Enhanced: What is the total premium generated by each agency in the Northeast region during the current year (2025)?

Original: Which agencies generated the most premium in my region this year?
Enhanced: "What is the total premium generated by each agency in the Northeast region during the current year (2025)?"

Original: How does the premium generated this year compare to last year?
Enhanced: "What is the total premium generated in the current year (2025) compared to the previous year (2024)?"

Original: What is the trend of premium generation over the last five years?
Enhanced: "What is the trend of premium generation over the last five years?"

Original: Which agencies have shown the most growth in premium generation?
Enhanced: "Which agencies ha